In [1]:

from jgtpy import JGTIDS as ids,JGTIDSRequest as RQ
from jgtpy import jgtapyhelper as tah
from jgtpy import JGTPDSP as pds
from jgtutils.jgtconstants import VOLUME



Here is the function I Coded myself to Adds MFI (Market Facilitation Index (reference: B. Williams)) squat column logics in order to validate and learn if the usage of the SQUAT in the last observed wave could help algorithm in decision to trade or not when we have trading signals.

```python

# MFI Constants for the column logics
from jgtutils.jgtconstants import MFI
MFI_SQUAT = "mfi_sq"
MFI_GREEN = "mfi_green"
MFI_FADE = "mfi_fade"
MFI_FAKE = "mfi_fake"
MFI_SIGNAL = "mfi_sig"
MFI_VAL = "mfiv"




def _ids_add_mfi_squat_column_logics_v1(dfsrc, quiet=False):
    """
    Adds MFI (Market Facilitation Index (reference: B. Williams)) squat column logics to the given DataFrame.

    Parameters:
    - dfsrc (DataFrame): The input DataFrame to which the MFI squat column logics will be added.
    - quiet (bool): If True, suppresses the output messages. Default is False.

    Returns:
    - dfsrc (DataFrame): The updated DataFrame with the MFI squat column logics added.
    """

    dfsrc[MFI_SQUAT] = (
        (dfsrc.Volume > 0)
        & (dfsrc.Volume > dfsrc.Volume.shift())
        & (dfsrc.mfi < dfsrc.mfi.shift())
    ).astype(int)
    dfsrc[MFI_GREEN] = (
        (dfsrc.Volume > 0)
        & (dfsrc.Volume > dfsrc.Volume.shift())
        & (dfsrc.mfi > dfsrc.mfi.shift())
    ).astype(int)
    dfsrc[MFI_FADE] = (
        (dfsrc.Volume > 0)
        & (dfsrc.Volume < dfsrc.Volume.shift())
        & (dfsrc.mfi < dfsrc.mfi.shift())
    ).astype(int)
    dfsrc[MFI_FAKE] = (
        (dfsrc.Volume > 0)
        & (dfsrc.Volume < dfsrc.Volume.shift())
        & (dfsrc.mfi > dfsrc.mfi.shift())
        ).astype(int)
    dfsrc[MFI_SIGNAL] = dfsrc.apply(
         lambda row: 4 if row[MFI_SQUAT] > 0 else 1 if row[MFI_GREEN] > 0 else 2 if row[MFI_FADE]>0 else 3 if row[MFI_FAKE]>0 else 0 , axis=1
     )
    dfsrc[MFI_VAL] = dfsrc.apply(
         lambda row: "++" if row[MFI_SIGNAL] == 1 else "--" if row[MFI_SIGNAL] == 2 else "-+" if row[MFI_SIGNAL] == 3 else "+-" if row[MFI_SIGNAL] == 4 else "0" , axis=1)
    return dfsrc





```
## Usage:

```python

from jgtpy import JGTIDSRequest as RQ
from jgtpy import jgtapyhelper as tah
from jgtpy import JGTPDSP as pds

## Get data
i="SPX500";t="D1"
dfsrc = pds.getPH(i,t,use_fresh=False)

#### Create our current Indicators


rq = RQ.JGTIDSRequest(i,t)
res_tah= tah.toids(dfsrc,rq=rq)


df_res_tah_with_mfi_sig=_ids_add_mfi_squat_column_logics_v1(res_tah)

```


You analyze the code, learn from it, suggest best practices or approve it and produce abstract, documentations etc to explain it.


### PURPOSE: Adequate Migration of IDS to TAH


## Get data

In [2]:

i="SPX500";t="D1"


In [3]:
# Compare the two datasets from ids.tocds(... ) and tah.toids(...)
## -->  MAKE A PLAN

In [4]:
dfsrc = pds.getPH(i,t,use_fresh=False)

_dt_requirements: 2024-06-17 21:20:59.260666 dt_crop_last: None
ldf: 424


In [5]:
dfsrc.head(2)


,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,High,Low,Close,Median
Date,,,,,,,,,,,,,,
2022-10-25 21:00:00,3843.96,3887.79,3814.59,3844.09,3844.36,3888.29,3815.19,3844.49,173950,3844.16,3888.04,3814.89,3844.29,3851.465
2022-10-26 21:00:00,3844.09,3861.63,3748.68,3782.67,3844.49,3862.13,3749.28,3783.07,221887,3844.29,3861.88,3748.98,3782.87,3805.430


In [6]:
dfsrc.tail(2)


,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,High,Low,Close,Median
Date,,,,,,,,,,,,,,
2024-06-13 21:00:00,5431.18,5437.32,5392.3,5429.92,5431.78,5437.92,5392.9,5430.32,126577,5431.48,5437.62,5392.6,5430.12,5415.11
2024-06-16 21:00:00,5429.92,5435.82,5428.3,5432.30,5430.32,5436.42,5428.9,5432.90,6777,5430.12,5436.12,5428.6,5432.60,5432.36


#### Create our current Indicators

In [7]:

rq = RQ.JGTIDSRequest(i,t)


In [8]:

res_jgtids = ids.tocds(dfsrc,rq=rq)


Tide Alligator has not enough bars in the DataFrame
 ADDING SQUAT PROTO Will be in CDS Module


In [9]:

# MFI Constants for the column logics
from jgtutils.jgtconstants import MFI,MFI_SQUAT,MFI_GREEN,MFI_FADE,MFI_FAKE,MFI_SIGNAL,MFI_VAL
# MFI_SQUAT = "mfi_sq"
# MFI_GREEN = "mfi_green"
# MFI_FADE = "mfi_fade"
# MFI_FAKE = "mfi_fake"
# MFI_SIGNAL = "mfi_sig"
# MFI_VAL = "mfi_str"


In [10]:


def _cds_add_mfi_squat_n_signals_column_logics_v1(dfsrc, quiet=False):
    """
    Adds MFI (Market Facilitation Index (reference: B. Williams)) squat column logics to the given DataFrame.

    Parameters:
    - dfsrc (DataFrame): The input DataFrame to which the MFI squat column logics will be added.
    - quiet (bool): If True, suppresses the output messages. Default is False.

    Returns:
    - dfsrc (DataFrame): The updated DataFrame with the MFI squat column logics added.
    """

    dfsrc[MFI_SQUAT] = (
        (dfsrc.Volume > 0)
        & (dfsrc.Volume > dfsrc.Volume.shift())
        & (dfsrc.mfi < dfsrc.mfi.shift())
    ).astype(int)
    dfsrc[MFI_GREEN] = (
        (dfsrc.Volume > 0)
        & (dfsrc.Volume > dfsrc.Volume.shift())
        & (dfsrc.mfi > dfsrc.mfi.shift())
    ).astype(int)
    dfsrc[MFI_FADE] = (
        (dfsrc.Volume > 0)
        & (dfsrc.Volume < dfsrc.Volume.shift())
        & (dfsrc.mfi < dfsrc.mfi.shift())
    ).astype(int)
    dfsrc[MFI_FAKE] = (
        (dfsrc.Volume > 0)
        & (dfsrc.Volume < dfsrc.Volume.shift())
        & (dfsrc.mfi > dfsrc.mfi.shift())
        ).astype(int)
    dfsrc[MFI_SIGNAL] = dfsrc.apply(
         lambda row: 4 if row[MFI_SQUAT] > 0 else 1 if row[MFI_GREEN] > 0 else 2 if row[MFI_FADE]>0 else 3 if row[MFI_FAKE]>0 else 0 , axis=1
     )
    dfsrc[MFI_VAL] = dfsrc.apply(
         lambda row: "++" if row[MFI_SIGNAL] == 1 else "--" if row[MFI_SIGNAL] == 2 else "-+" if row[MFI_SIGNAL] == 3 else "+-" if row[MFI_SIGNAL] == 4 else "0" , axis=1)
    return dfsrc



In [11]:
with_squat=_cds_add_mfi_squat_n_signals_column_logics_v1(res_jgtids).copy()

In [12]:
with_squat.tail(2)


,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,...,price_peak_above,price_peak_bellow,ao_peak_above,ao_peak_bellow,mfi_sq,mfi_green,mfi_fade,mfi_fake,mfi_sig,mfi_str
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-13 21:00:00,5431.18,5437.32,5392.3,5429.92,5431.78,5437.92,5392.9,5430.32,126577,5431.48,...,0,0,0,0,1,0,0,0,4,+-
2024-06-16 21:00:00,5429.92,5435.82,5428.3,5432.30,5430.32,5436.42,5428.9,5432.90,6777,5430.12,...,0,0,0,0,0,0,0,1,3,-+


In [13]:

col_selected=[VOLUME, MFI_VAL, MFI, MFI_SQUAT, MFI_GREEN, MFI_FADE, MFI_FAKE, MFI_SIGNAL]
observed_mfi_n_sig=with_squat[col_selected]

In [14]:
observed_mfi_n_sig

,Volume,mfi_str,mfi,mfi_sq,mfi_green,mfi_fade,mfi_fake,mfi_sig
Date,,,,,,,,
2023-05-17 21:00:00,112680,0,48.375932,0,0,0,0,0
2023-05-18 21:00:00,142228,+-,24.798211,1,0,0,0,4
2023-05-21 21:00:00,100215,-+,34.865040,0,0,0,1,3
2023-05-22 21:00:00,109892,++,63.262112,0,1,0,0,1
2023-05-23 21:00:00,124749,+-,41.691717,1,0,0,0,4
...,...,...,...,...,...,...,...,...
2024-06-10 21:00:00,83933,-+,61.656321,0,0,0,1,3
2024-06-11 21:00:00,135387,+-,55.027440,1,0,0,0,4
2024-06-12 21:00:00,102589,--,43.152775,0,0,1,0,2


## BELLOW CONFIRM What is Expected


* JGTIDS (which is more than just IDS) has 60 columns
* TAH (jgtapyhelper) has 41 columns, just IDS

#### In summary

* JGTIDS with ids.tocds(...) GENERATES SIGNALS

----


#### TOCDS (JGTIDS) Signal Columns

fdbb
fdbs
fdb
aoaz
aobz
zlc
zlcb
zlcs
zcol
sz
bz
acs
acb
ss
sb
price_peak_above
price_peak_bellow
ao_peak_above
ao_peak_bellow

----


In [15]:
res_tah= tah.toids(dfsrc,rq=rq)

Tide Alligator has not enough bars in the DataFrame
 ADDING SQUAT PROTO Will be in CDS Module


In [16]:


df_res_tah_with_mfi_sig=_cds_add_mfi_squat_n_signals_column_logics_v1(res_tah)
res_tah_with_mfi_sig=df_res_tah_with_mfi_sig.copy()
observed_mfi_n_sig_tah=res_tah_with_mfi_sig[col_selected]


In [17]:

observed_mfi_n_sig_tah.tail(2)


,Volume,mfi_str,mfi,mfi_sq,mfi_green,mfi_fade,mfi_fake,mfi_sig
Date,,,,,,,,
2024-06-13 21:00:00,126577,+-,35.567283,1,0,0,0,4
2024-06-16 21:00:00,6777,-+,110.963553,0,0,0,1,3


In [18]:
res_tah.tail(2)

,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,...,fl55,fh89,fl89,mfi,mfi_sq,mfi_green,mfi_fade,mfi_fake,mfi_sig,mfi_str
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-13 21:00:00,5431.18,5437.32,5392.3,5429.92,5431.78,5437.92,5392.9,5430.32,126577,5431.48,...,0,0,0,35.567283,1,0,0,0,4,+-
2024-06-16 21:00:00,5429.92,5435.82,5428.3,5432.30,5430.32,5436.42,5428.9,5432.90,6777,5430.12,...,0,0,0,110.963553,0,0,0,1,3,-+


In [19]:
res_jgtids.tail(2)

,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,...,price_peak_above,price_peak_bellow,ao_peak_above,ao_peak_bellow,mfi_sq,mfi_green,mfi_fade,mfi_fake,mfi_sig,mfi_str
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-13 21:00:00,5431.18,5437.32,5392.3,5429.92,5431.78,5437.92,5392.9,5430.32,126577,5431.48,...,0,0,0,0,1,0,0,0,4,+-
2024-06-16 21:00:00,5429.92,5435.82,5428.3,5432.30,5430.32,5436.42,5428.9,5432.90,6777,5430.12,...,0,0,0,0,0,0,0,1,3,-+


In [20]:
res_jgtids.columns


Index(['BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskHigh',
       'AskLow', 'AskClose', 'Volume', 'Open', 'High', 'Low', 'Close',
       'Median', 'ao', 'ac', 'jaw', 'teeth', 'lips', 'bjaw', 'bteeth', 'blips',
       'fh', 'fl', 'fh3', 'fl3', 'fh5', 'fl5', 'fh8', 'fl8', 'fh13', 'fl13',
       'fh21', 'fl21', 'fh34', 'fl34', 'fh55', 'fl55', 'fh89', 'fl89', 'mfi',
       'fdbb', 'fdbs', 'fdb', 'aoaz', 'aobz', 'zlc', 'zlcb', 'zlcs', 'zcol',
       'sz', 'bz', 'acs', 'acb', 'ss', 'sb', 'price_peak_above',
       'price_peak_bellow', 'ao_peak_above', 'ao_peak_bellow', 'mfi_sq',
       'mfi_green', 'mfi_fade', 'mfi_fake', 'mfi_sig', 'mfi_str'],
      dtype='object')

In [21]:
res_tah.columns

Index(['BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskHigh',
       'AskLow', 'AskClose', 'Volume', 'Open', 'High', 'Low', 'Close',
       'Median', 'ao', 'ac', 'jaw', 'teeth', 'lips', 'bjaw', 'bteeth', 'blips',
       'fh', 'fl', 'fh3', 'fl3', 'fh5', 'fl5', 'fh8', 'fl8', 'fh13', 'fl13',
       'fh21', 'fl21', 'fh34', 'fl34', 'fh55', 'fl55', 'fh89', 'fl89', 'mfi',
       'mfi_sq', 'mfi_green', 'mfi_fade', 'mfi_fake', 'mfi_sig', 'mfi_str'],
      dtype='object')

In [22]:

# Compare the two datasets from ids.tocds(... ) and tah.toids(...)
## List of columns that are different
for c in res_jgtids.columns:
    if c not in res_tah.columns:
        print(c)


fdbb
fdbs
fdb
aoaz
aobz
zlc
zlcb
zlcs
zcol
sz
bz
acs
acb
ss
sb
price_peak_above
price_peak_bellow
ao_peak_above
ao_peak_bellow
